## Загрузка данных

Скачайте сырые данные о поездках жёлтого такси с сайта [TLC](http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml)

**Поездки зелёного такси и лимузинов нас не интересуют!**

Данные выложены в файлах по месяцам. Скачайте так много данных жёлтого такси, как сможете; чем больше вы будете использовать данных, тем точнее получатся ваши прогнозы. Если вы решите использовать не все данные, а только часть, выбирайте её по времени с конца. Абсолютный минимум необходимых данных — 6 месяцев: один, последний месяц, вам понадобится для тестирования, предшествующие 5 — для обучения. По 5 месяцам можно построить прогнозы, учитывающие дневную и недельную сезонности, но в данных есть и годовая. Чтобы её учесть, необходимы данные как минимум за 2 года.

## Обработка данных

Обработайте сырые данные по следующей схеме.

1. Почистите данные от ошибок и аномалий.
2. Отбросьте минуты и секунды во времени начала поездки.
3. Нью-Йорк вписан в прямоугольник от **-74.25559** до **-73.70001** градусов долготы и от **40.49612** до **40.91553** широты. Разбейте этот прямоугольник на **2500** одинаковых прямоугольных областей — по **50** интервалов вдоль каждой оси.
4. Посчитайте количество поездок за каждый час из каждой области. Не забудьте, что если в сырых данных для какой-то пары час-область нет ни одной записи, то в агрегированных данных для неё должен стоять 0.

В дальнейшем, когда вы будете предсказывать получившиеся ряды, нужно будет загружать ваши прогнозы на kaggle, поэтому нужно, чтобы идентификаторы ячеек были определены однозначно. В файле **regions.csv** даны идентификаторы ячеек, которые вам нужно использовать, и географические координаты их границ.

Для сдачи задания этой недели вам нужно агрегировать только данные за **май 2016**, но, когда задание будет сдано, не забудьте запустить ваш агрегирующий скрипт на всех остальных месяцах, которые вы собираетесь использовать при прогнозировании.

## Загрузка данных

Загрузка данных за май 2016 года, и преобразование строк с датами и временем к типу **datetime**

In [1]:
import numpy as np
import pandas as pd

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def read_data(name, state):
    columns = []
    if state == "bad":
        columns = ["VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count",
               "trip_distance", "RatecodeID", "store_and_fwd_flag", "PULocationID", "DOLocationID",
               "payment_type", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount",
               "improvement_surcharge", "total_amount", "", ""]
    else:
        columns = ["vendor_id", "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count",
               "trip_distance", "pickup_longitude", 'pickup_latitude', "rate_code", "store_and_fwd_flag",
               "dropoff_longitude", "dropoff_latitude", "payment_type", "fare_amount",
               "surcharge", "mta_tax", 'tip_amount', "tolls_amount", "total_amount"]
        
    parse_dates = ["tpep_pickup_datetime", "tpep_dropoff_datetime"]
    data = {
    'normal': lambda name: pd.read_csv("data/" + name, header=0,
                   sep=",", parse_dates=parse_dates),
    'bad': lambda name: pd.read_csv("data/" + name, sep=",", names=columns,
                   skiprows=1, parse_dates=parse_dates),
    'old': lambda name: pd.read_csv("data/" + name, skiprows=1,
                   sep=",", parse_dates=parse_dates, names=columns)
    }[state](name)
    
    return data

## Фильтрация данных

Провести фильтрацию: удалить поездки с
- нулевой длительностью
- нулевым количеством пассажиров
- нулевым расстоянием поездки по счётчику
- координатами начала, не попадающими в прямоугольник Нью-Йорка

Можно придумать ещё какие-то критерии для фильтрации данных; тем не менее, не стоит применять дополнительные фильтры: начиная с четвёртой недели вам предстоит сравнивать качество ваших прогнозов с качеством прогнозов других слушателей, и, чтобы это сравнение было корректным, нужно, чтобы данные у всех были предобработаны одинаково.

In [4]:
# Фильтрация данных и отброс минут и секунд

def filter_data(data, _type):
    # Фильтрация данных с нулевой длительностью поездки
    data = data[data.tpep_pickup_datetime != data.tpep_dropoff_datetime]
    data = data[data.passenger_count != 0] # Фильтрация данных с нулевым количеством пассажиров
    data = data[data.trip_distance != 0.0] # Фильтрация данных с нулевым расстоянием по счетчику
    
     # Отброс минут и секунд
    data.tpep_pickup_datetime = map(lambda x: x.replace(minute=0, second=0), data.tpep_pickup_datetime)
    
    if _type == "bad":
        return data
    
    # Фильтрация поездок, которые находятся за пределами Нью-Йорка
    long_compare = (data.pickup_longitude >= -74.25559) & (data.pickup_longitude <= -73.70001)
    lat_compare = (data.pickup_latitude >= 40.49612) & (data.pickup_latitude <= 40.91553)
    data = data[long_compare & lat_compare]
    
    return data

## Агрегация данных

Агрегируйте данные за май 2016 года по часам и областям (можно использовать функцию stats.binned_statistic_2d, с которой вы сталкивались в заданиях четвёртого курса). Не забудьте проверить, что идентификаторы, которые вы присваиваете ячейкам, соответствуют приложенному выше файлу.

In [5]:
from scipy.stats import binned_statistic_2d

In [6]:
# Обёртка для функции binned_statistic_2d, для получения более точного номера области,
# в соответствии с приложенным к заданию файлом

def bin_stats(input_data):
    s, x_edge, y_edge, binnumber = binned_statistic_2d(input_data.pickup_longitude,
                                                   input_data.pickup_latitude,
                                                   None, statistic='count', bins=50,
                                                   range=[[-74.25559, -73.70001], [40.49612, 40.91553]],
                                                  expand_binnumbers=True)
    return s, x_edge, y_edge, 50 * (binnumber[0] - 1) + binnumber[1]

In [7]:
import calendar
import itertools

In [8]:
def date_num_from_name(name):
    text = name.replace("_", " ").replace("-", " ").replace(".", " ")
    digit_array = [int(s) for s in text.split() if s.isdigit()]
    year = digit_array[0]
    month = digit_array[1]
    return year, month

In [9]:
# Создание пустой таблицы со всеми возможными значениями времени и регионов

def create_empty_frame(name):
    year, month = date_num_from_name(name)
    days = calendar.monthrange(year, month)[1]
    regions = range(1, 2501)
    start_date = str(month) + '/1/' + str(year)
    date = pd.date_range(start_date, periods=days*24, freq='H')
    agg_data = map(lambda x: (x[0], x[1]), itertools.product(regions, date))
    agg_data = pd.DataFrame(agg_data, columns = ['region', 'time'])
    return agg_data

In [10]:
def merge_agg_data(agg_data, data):
    # Получение количества поездок по области и времени заказа
    reg_trips = data[["region", "tpep_pickup_datetime", "trip_distance"]].groupby(["region", "tpep_pickup_datetime"], as_index=False).count()
    reg_trips.columns = ["region", "time", "trips"]
    
    # Набор агрегированных данных по областям и временем поездок на жёлтом такси
    agg_data = pd.merge(agg_data, reg_trips, how='left').fillna(0)
    return agg_data

In [11]:
# Весь отработанный процесс фильтрации, агрегации и сохранение данных

def aggregate_data(name, _type):
    flag = {
        'bad': True,
        'normal': True,
        'old': True
    }.get(_type, False)
    
    if flag is False:
        print "Set _type is normal, bad or old"
        return
    
    data = read_data(name, _type)
    data = filter_data(data, _type)
    
    if _type != "bad":
        s, x_edge, y_edge, regions = bin_stats(data)
        data["region"] = regions
    else:
        data["region"] = data["PULocationID"]
    
    agg_data = create_empty_frame(name)
    agg_data = merge_agg_data(agg_data, data)
    
    # Сохранение нового представления данных о поездках в csv
    agg_data.to_csv("u_data/" + name, sep=",", index=False)
    
    os.remove("data/" + name)
    return agg_data

Постройте график количества поездок жёлтого такси из ячейки, содержащей Empire State Building.

In [9]:
regions = pd.read_csv("regions.csv", sep=";")
regions.head(5)

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


In [7]:
x_edge = pd.unique(np.concatenate((regions.west, regions.east)))
y_edge = pd.unique(np.concatenate((regions.south, regions.north)))

In [8]:
# Нахождение области, в котором находится Empire State Building по заранее известным координатам

x = -73.985428
y = 40.748817

x_num = 0
for i in xrange(len(x_edge) - 1):
    if x >= x_edge[i] and x <= x_edge[i + 1]:
        x_num = i;
        break
    
y_num = 0
for i in xrange(len(y_edge) - 1):
    if y >= y_edge[i] and y <= y_edge[i + 1]:
        y_num = i;
        break

reg_index = 50 * x_num + y_num + 1
print "Empire State Building located in %d region." % (reg_index)

Empire State Building located in 1231 region.


In [ ]:
# Почасовая отчетность по поездкам в данной области
esb_reg_data = agg_data[agg_data.region == reg_index]
print esb_reg_data

In [ ]:
pylab.figure(figsize=(14,8))
pylab.plot(esb_reg_data.time, esb_reg_data.trips)
pylab.title("Empire State Building trips", fontweight='bold')
pylab.xlabel("Time", fontweight='bold')
pylab.ylabel("Trips", fontweight='bold')
pylab.show()

Посчитайте, сколько в мае 2016 было пар час-ячейка, для которых не было совершено ни одной поездки.

In [ ]:
zeros_pair = agg_data[agg_data.trips == 0].shape[0]
all_values = agg_data.shape[0]
print "%d out of %d pairs hour-region do not have trips" % (zeros_pair, all_values)

## Подготовка остальных данных

Получение списка всех файлов с данными о поездках.

In [12]:
import os
data_list = os.listdir("data")
data_list

['yellow_tripdata_2009-01.csv',
 'yellow_tripdata_2009-02.csv',
 'yellow_tripdata_2009-03.csv',
 'yellow_tripdata_2009-04.csv',
 'yellow_tripdata_2009-05.csv',
 'yellow_tripdata_2009-06.csv']

In [13]:
print "Begin..."
for fname in data_list:
    aggregate_data(fname, "old")
    print fname + " is complete!"
print "...End"

Begin...
yellow_tripdata_2009-01.csv is complete!
yellow_tripdata_2009-02.csv is complete!
yellow_tripdata_2009-03.csv is complete!
yellow_tripdata_2009-04.csv is complete!
yellow_tripdata_2009-05.csv is complete!
yellow_tripdata_2009-06.csv is complete!
...End
